In [5]:
import sys
sys.path.insert(0, '/cndd/fangming/CEMBA/snmcseq_dev')
import importlib

from __init__ import *
from __init__jupyterlab import *
import snmcseq_utils
importlib.reload(snmcseq_utils)
import CEMBA_preproc_utils
importlib.reload(CEMBA_preproc_utils)
import CEMBA_clst_utils

from scipy.io import mmread
from scipy import sparse
import time

import fbpca

8 datasets (modalities)
- Expression values (raw or normalized) in anndata.X
- At least one embedding (e.g., tSNE, UMAP) in anndata.obsm, specified with the prefix X_ (e.g., by default scanpy stores UMAP coordinates in anndata.obsm['X_umap'])
- A unique identifier for every cell is available in an anndata.obs field (you can specify this with the --obs-names option)
- A unique identifier for every gene is available in an anndata.var field (you can specify which field to use with the --var-names option)

In [6]:
data_dir = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons'
sys.path.insert(0, data_dir)
from __init__datasets import *

In [8]:
mods

('snmcseq_gene',
 'snatac_gene',
 'smarter_cells',
 'smarter_nuclei',
 '10x_cells_v2',
 '10x_cells_v3',
 '10x_nuclei_v3',
 '10x_nuclei_v3_macosko')

### Getting expression matrices 

In [30]:
# for expression matrices

data_dict = {}

for mod in mods:
    if mod == 'smarter_cells':
        print(mod)
        gene_file = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/{}_features.gene'.format(mod)
        cell_file = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/{}_features.cell'.format(mod)
        mtx_file = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/{}_features.npz'.format(mod)
        gene_by_cell_matrix = snmcseq_utils.load_gc_matrix(gene_file, cell_file, mtx_file)
        genes = gene_by_cell_matrix.gene
        cells = gene_by_cell_matrix.cell
        mat = gene_by_cell_matrix.data
        df_gc_matrix = pd.DataFrame(mat.todense(), index=genes, columns=cells)
        data_dict[mod] = df_gc_matrix
        
    if mod == 'snmcseq_gene': # dna methylation
        file = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/{}_features.tsv'.format(mod)
        df_gc_matrix = pd.read_csv(file, sep='\t', index_col=0)
        data_dict[mod] = df_gc_matrix
        
        
    

smarter_cells


### Getting metadata  

In [10]:
# coordinates; cluster labels
f = '/cndd/fangming/CEMBA/data/MOp_all/results_final/miniatlas_fig4_scf_clusterings.tsv'
meta = pd.read_csv(f, sep='\t')
print(meta.shape)
meta.head()

(408885, 11)


/cndd/fangming/venvs/conda_dobro/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,sample,joint_embedding_x,joint_embedding_y,modality,modality_name,single_modality_cluster,single_modality_annot,joint_cluster_round1,joint_cluster_round2,joint_cluster_round3,joint_cluster_round4
0,snmcseq_gene_2C_M_0,0.471629,-3.134735,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Tenm2,L4-IT-Rorb_Rorb-Tenm2,1,1-1,1-1-1,1-1-1-1
1,snmcseq_gene_2C_M_1,-2.414973,-12.098081,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,4,4-2,4-2-1,4-2-1-1
2,snmcseq_gene_2C_M_100,-1.446751,-12.051894,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,4,4-1,4-1-1,4-1-1-1
3,snmcseq_gene_2C_M_1000,-1.901734,-13.660591,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,4,4-2,4-2-1,4-2-1-1
4,snmcseq_gene_2C_M_1001,-0.981086,-11.936621,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,4,4-1,4-1-1,4-1-1-1


In [13]:
f = '/cndd/fangming/CEMBA/data/MOp_all/results_final/miniatlas_fig4_scf_annotation_round1.tsv'
annot1 = pd.read_csv(f, sep='\t')
print(annot1.shape)
annot1.head()

f = '/cndd/fangming/CEMBA/data/MOp_all/results_final/miniatlas_fig4_scf_annotation_round2.tsv'
annot2 = pd.read_csv(f, sep='\t')
print(annot2.shape)
annot2.head()

f = '/cndd/fangming/CEMBA/data/MOp_all/results_final/miniatlas_fig4_scf_annotation_round3.tsv'
annot3 = pd.read_csv(f, sep='\t')
print(annot3.shape)
annot3.head()

(10, 3)
(29, 3)
(56, 3)


,cluster_id_iterative,annotation_curated,color
0,1-1-1,L4/5 IT (1),#3CBC78
1,1-2-1,L4/5 IT (2),#3CBC45
2,1-2-2,L4/5 IT (3),#5DDB65
3,2-1-1,L6 CT Cpa6 (1),#338C5E
4,2-1-2,L6 CT Cpa6 (2),#338C5E


In [17]:
tmp = pd.merge(meta, 
             annot1.rename(columns={'annotation_curated': 'joint_cluster_round1_annotation'})
             [['cluster_id_iterative', 'joint_cluster_round1_annotation']], 
             left_on='joint_cluster_round1', right_on='cluster_id_iterative').drop('cluster_id_iterative', axis=1)
tmp.head()

,sample,joint_embedding_x,joint_embedding_y,modality,modality_name,single_modality_cluster,single_modality_annot,joint_cluster_round1,joint_cluster_round2,joint_cluster_round3,joint_cluster_round4,joint_cluster_round1_annotation
0,snmcseq_gene_2C_M_0,0.471629,-3.134735,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Tenm2,L4-IT-Rorb_Rorb-Tenm2,1,1-1,1-1-1,1-1-1-1,L4/5 IT
1,snmcseq_gene_2C_M_1006,4.204999,-2.770652,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Ryr3,L4-IT-Rorb_Rorb-Ryr3,1,1-2,1-2-1,1-2-1-1,L4/5 IT
2,snmcseq_gene_2C_M_1009,9.292183,-5.484662,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Cpne4,L4-IT-Rorb_Rorb-Cpne4,1,1-2,1-2-2,1-2-2-3,L4/5 IT
3,snmcseq_gene_2C_M_101,5.540587,-2.440171,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Cpne4,L4-IT-Rorb_Rorb-Cpne4,1,1-2,1-2-2,1-2-2-6,L4/5 IT
4,snmcseq_gene_2C_M_1011,-0.091199,-2.604921,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Tenm2,L4-IT-Rorb_Rorb-Tenm2,1,1-1,1-1-1,1-1-1-1,L4/5 IT


In [19]:
tmp = pd.merge(tmp, 
             annot2.rename(columns={'annotation_curated': 'joint_cluster_round2_annotation'})
             [['cluster_id_iterative', 'joint_cluster_round2_annotation']], 
             left_on='joint_cluster_round2', right_on='cluster_id_iterative').drop('cluster_id_iterative', axis=1)
tmp.head()

,sample,joint_embedding_x,joint_embedding_y,modality,modality_name,single_modality_cluster,single_modality_annot,joint_cluster_round1,joint_cluster_round2,joint_cluster_round3,joint_cluster_round4,joint_cluster_round1_annotation,joint_cluster_round2_annotation
0,snmcseq_gene_2C_M_0,0.471629,-3.134735,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Tenm2,L4-IT-Rorb_Rorb-Tenm2,1,1-1,1-1-1,1-1-1-1,L4/5 IT,L4/5 IT (1)
1,snmcseq_gene_2C_M_1011,-0.091199,-2.604921,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Tenm2,L4-IT-Rorb_Rorb-Tenm2,1,1-1,1-1-1,1-1-1-1,L4/5 IT,L4/5 IT (1)
2,snmcseq_gene_2C_M_1012,-0.037576,-8.106225,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,1,1-1,1-1-1,1-1-1-1,L4/5 IT,L4/5 IT (1)
3,snmcseq_gene_2C_M_1022,0.906631,-0.785923,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Cpne4,L4-IT-Rorb_Rorb-Cpne4,1,1-1,1-1-1,1-1-1-1,L4/5 IT,L4/5 IT (1)
4,snmcseq_gene_2C_M_1024,1.627603,-1.945875,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Cpne4,L4-IT-Rorb_Rorb-Cpne4,1,1-1,1-1-1,1-1-1-1,L4/5 IT,L4/5 IT (1)


In [20]:
tmp = pd.merge(tmp, 
             annot3.rename(columns={'annotation_curated': 'joint_cluster_round3_annotation'})
             [['cluster_id_iterative', 'joint_cluster_round3_annotation']], 
             left_on='joint_cluster_round3', right_on='cluster_id_iterative').drop('cluster_id_iterative', axis=1)
tmp.head()

,sample,joint_embedding_x,joint_embedding_y,modality,modality_name,single_modality_cluster,single_modality_annot,joint_cluster_round1,joint_cluster_round2,joint_cluster_round3,joint_cluster_round4,joint_cluster_round1_annotation,joint_cluster_round2_annotation,joint_cluster_round3_annotation
0,snmcseq_gene_2C_M_0,0.471629,-3.134735,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Tenm2,L4-IT-Rorb_Rorb-Tenm2,1,1-1,1-1-1,1-1-1-1,L4/5 IT,L4/5 IT (1),L4/5 IT (1)
1,snmcseq_gene_2C_M_1011,-0.091199,-2.604921,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Tenm2,L4-IT-Rorb_Rorb-Tenm2,1,1-1,1-1-1,1-1-1-1,L4/5 IT,L4/5 IT (1),L4/5 IT (1)
2,snmcseq_gene_2C_M_1012,-0.037576,-8.106225,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,1,1-1,1-1-1,1-1-1-1,L4/5 IT,L4/5 IT (1),L4/5 IT (1)
3,snmcseq_gene_2C_M_1022,0.906631,-0.785923,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Cpne4,L4-IT-Rorb_Rorb-Cpne4,1,1-1,1-1-1,1-1-1-1,L4/5 IT,L4/5 IT (1),L4/5 IT (1)
4,snmcseq_gene_2C_M_1024,1.627603,-1.945875,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Cpne4,L4-IT-Rorb_Rorb-Cpne4,1,1-1,1-1-1,1-1-1-1,L4/5 IT,L4/5 IT (1),L4/5 IT (1)


In [21]:
output = '/cndd2/portal_dev/metadata_for_cellxgene_8datasets.tsv'
tmp.to_csv(output, sep='\t', header=True, index=False, na_rep='NA')